In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import gmaps
from citipy import citipy
from scipy.stats import linregress

In [2]:
import os
from dotenv import load_dotenv
# Load .env enviroment variables into the notebook
load_dotenv()
# Get the API key from the environment variable and store as Python variable
gkey = os.getenv("gkey")
# Configure gmaps
gmaps.configure(api_key=gkey)

In [13]:
quake_df = pd.read_csv("../data/all_earthquake_data2.zip")

quake_df.head()

,time,latitude,longitude,mag,place
0,1969-12-13T18:59:11.430Z,19.313000,-155.104000,2.78,"17 km S of Fern Forest, Hawaii"
1,1969-12-13T17:55:57.400Z,46.709000,-119.234333,1.60,"13 km SSW of Othello, Washington"
2,1969-12-13T13:03:04.570Z,19.466667,-155.487667,1.94,"26 km W of Volcano, Hawaii"
3,1969-12-13T12:53:50.580Z,19.443167,-155.248833,NaN,"1 km W of Volcano, Hawaii"
4,1969-12-13T10:43:38.000Z,19.304167,-155.210500,2.51,"15 km S of Volcano, Hawaii"


In [14]:
# Store latitude and longitude in locations
quake_df['year'] = pd.DatetimeIndex(quake_df['time']).year
locations = quake_df[["latitude", "longitude"]]
magnitude  = quake_df[['mag']].astype(float)
quake_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4053081 entries, 0 to 4053080
Data columns (total 6 columns):
 #   Column     Dtype  
---  ------     -----  
 0   time       object 
 1   latitude   float64
 2   longitude  float64
 3   mag        float64
 4   place      object 
 5   year       int64  
dtypes: float64(3), int64(1), object(2)
memory usage: 185.5+ MB


In [9]:
# Plot Heatmap
fig = gmaps.figure(zoom_level=2, center=[31.9686, -99.9018])



# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = quake_df['mag'] > 3,
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [15]:
num_of_quakes_by_yr = quake_df['year'].value_counts()
num_of_quakes_by_yr.sort_index()

1969       400
1970      5657
1971      8672
1972      4957
1973      9994
1974     15139
1975     20377
1976     17975
1977     19608
1978     21420
1979     28231
1980     29391
1981     38819
1982     42401
1983     55344
1984     62469
1985     59094
1986     64769
1987     46523
1988     46911
1989     62091
1990     60454
1991     55425
1992     98341
1993     75120
1994     73237
1995     72225
1996     66105
1997     75361
1998     72315
1999     73050
2000     76297
2001     77089
2002     99451
2003    104090
2004    123003
2005    113412
2006    105501
2007     98301
2008    129320
2009     92061
2010    126845
2011    110971
2012    109522
2013    118764
2014    139921
2015    127072
2016    126595
2017    134606
2018    188723
2019    183415
2020    192297
2021     93950
Name: year, dtype: int64

In [16]:
quake_df['decade'] = quake_df['year'].apply(lambda x: str(x) [:3] + '0s')
quake_df

,time,latitude,longitude,mag,place,year,decade
0,1969-12-13T18:59:11.430Z,19.313000,-155.104000,2.78,"17 km S of Fern Forest, Hawaii",1969,1960s
1,1969-12-13T17:55:57.400Z,46.709000,-119.234333,1.60,"13 km SSW of Othello, Washington",1969,1960s
2,1969-12-13T13:03:04.570Z,19.466667,-155.487667,1.94,"26 km W of Volcano, Hawaii",1969,1960s
3,1969-12-13T12:53:50.580Z,19.443167,-155.248833,NaN,"1 km W of Volcano, Hawaii",1969,1960s
4,1969-12-13T10:43:38.000Z,19.304167,-155.210500,2.51,"15 km S of Volcano, Hawaii",1969,1960s
...,...,...,...,...,...,...,...
4053076,2021-07-25T00:14:10.316Z,62.637400,-150.151200,1.30,"21 km N of Chase, Alaska",2021,2020s
4053077,2021-07-25T00:05:08.190Z,19.228500,-155.453659,1.84,"3 km NE of P?hala, Hawaii",2021,2020s
4053078,2021-07-25T00:03:53.992Z,11.217900,-86.871700,4.50,"74 km SSW of Masachapa, Nicaragua",2021,2020s
4053079,2021-07-25T00:02:53.559Z,60.157700,-141.588500,1.10,"124 km WNW of Yakutat, Alaska",2021,2020s


In [21]:
seventies = quake_df.loc[(quake_df['decade'] == '1970s') & (quake_df['mag'] >= 3)]
eighties = quake_df.loc[(quake_df['decade'] == '1980s') & (quake_df['mag'] >= 3)]
nineties = quake_df.loc[(quake_df['decade'] == '1990s') & (quake_df['mag'] >= 3)]
y2k = quake_df.loc[(quake_df['decade'] == '2000s') & (quake_df['mag'] >= 3)]
y2k1 = quake_df.loc[(quake_df['decade'] == '2010s') & (quake_df['mag'] >= 3)]
y2k2 = quake_df.loc[(quake_df['year'] == 2020) & (quake_df['mag'] >= 3)]


In [23]:
num_of_quakes_by_dec = quake_df['decade'].value_counts()
num_of_quakes_by_dec.sort_index()

1960s        400
1970s     152030
1980s     507812
1990s     721633
2000s    1018525
2010s    1366434
2020s     286247
Name: decade, dtype: int64

In [25]:
top10_quakes_all = quake_df.nlargest(10, 'mag')
top10_70s = seventies.nlargest(10, 'mag')
top10

,time,latitude,longitude,mag,place,year,decade
1862053,2004-12-26T00:58:53.450Z,3.295,95.982,9.1,2004 Sumatra - Andaman Islands Earthquake,2004,2000s
2546764,2011-03-11T05:46:24.120Z,38.297,142.373,9.1,"2011 Great Tohoku Earthquake, Japan",2011,2010s
2416376,2010-02-27T06:34:11.530Z,-36.122,-72.898,8.8,"36 km WNW of Quirihue, Chile",2010,2010s
1890830,2005-03-28T16:09:36.530Z,2.085,97.108,8.6,"78 km WSW of Singkil, Indonesia",2005,2000s
2669134,2012-04-11T08:38:36.720Z,2.327,93.063,8.6,off the west coast of northern Sumatra,2012,2010s
1493118,2001-06-23T20:33:14.130Z,-16.265,-73.641,8.4,"6 km SSW of Atico, Peru",2001,2000s
2149271,2007-09-12T11:10:26.830Z,-4.438,101.367,8.4,"122 km SW of Bengkulu, Indonesia",2007,2000s
93691,1977-08-19T06:08:55.000Z,-11.164,118.378,8.3,"265 km SW of Waingapu, Indonesia",1977,1970s
1007547,1994-10-04T13:22:55.840Z,43.773,147.321,8.3,"48 km E of Shikotan, Russia",1994,1990s
2069722,2006-11-15T11:14:13.570Z,46.592,153.266,8.3,NaN,2006,2000s
